In [10]:
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [12]:
data = pd.read_csv('../data/model_training_data.csv')
X, y = data[['Sex', 'Age', 'BodyweightKg', 'Best3SquatKg', 'Best3BenchKg']], data['Best3DeadliftKg']


In [14]:
data[['Sex']]

,Sex
0,M
1,M
2,F
3,F
4,M
...,...
93451,M
93452,M
93453,F
93454,F
